In [1]:
!pip install --upgrade gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip install groq --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.7 MB/s eta 0:00:00


In [3]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=0202c38124096115a3fb5d4dee0e4823474536f2bae142d94253c634295b4fc6
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [9]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.5 MB/s eta 0:00:00


In [24]:
import gradio as gr
import numpy as np
import joblib
import tempfile
import matplotlib.pyplot as plt
from google.colab import userdata
from groq import Groq

# Load model and scaler
scaler = joblib.load("/content/drive/MyDrive/Project/Model/scaler.joblib")
model = joblib.load("/content/drive/MyDrive/Project/Model/final_rf_model.joblib")

# Load Groq API key from Colab secrets
api_key = userdata.get('GrokAPi')
client = Groq(api_key=api_key)

def build_prompt(info, has_disease=True):
    context = "has been diagnosed with NAFLD." if has_disease else "does not currently have NAFLD, but has submitted liver-related test results for general liver health screening."
    return f"""
You are a licensed medical assistant specializing in liver disease.

A patient {context} Below is their profile with recent liver function test values.

---

### 🎯 **Instructions:**
Your response should be clear, visually structured, and informative. Format it using Markdown. Use bold, bullet points, numbered lists, and emojis to enhance clarity.

**You must include the following sections:**

#### 🧾 **Liver Profile Summary**
- Briefly highlight abnormal or notable values.
- Use correct units (mg/dL, IU/L, etc).

#### 🩺 **Interpretation**
- Explain what the values suggest about liver health.
- Mention possible causes or red flags.

#### 🥗 **Lifestyle & Dietary Recommendations**
1. 🍽️ **Diet**: Suggest foods to increase or avoid, with examples.
2. 🏃 **Physical Activity**: Recommend exercise patterns.
3. 💧 **Hydration & Sleep**: Best practices for recovery and balance.
4. 🧘 **Stress & Alcohol Management**: Coping strategies and cautionary notes.
5. ✅ **Follow-up**: When to revisit or test again.

### 🖼️ **Style Guidelines:**
- Use **clear headings with emojis**.
- Write in a **concise, supportive tone**.
- **Avoid tables** and complex jargon.
- You may use **ASCII symbols** or horizontal bars for emphasis.

---

### 🧬 **Patient Profile:**
- Age: {info['age']}
- Gender: {'Male' if info['gender'] == 1 else 'Female'}
- Total Bilirubin: {info['tb']} mg/dL
- Direct Bilirubin: {info['db']} mg/dL
- Alkaline Phosphotase: {info['alk_phos']} IU/L
- ALT (Alanine Aminotransferase): {info['alt']} U/L
- AST (Aspartate Aminotransferase): {info['ast']} U/L
- Total Proteins: {info['proteins']} g/dL
- Albumin: {info['albumin']} g/dL
- A/G Ratio: {info['ag_ratio']}

📌 Keep the tone professional, empathetic, and practical.
"""

def get_recommendation(info, has_disease):
    prompt = build_prompt(info, has_disease)
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are a medical assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def generate_liver_plot(info):
    features = ['tb', 'db', 'alk_phos', 'alt', 'ast', 'proteins', 'albumin', 'ag_ratio']
    values = [info[k] for k in features]
    ref_values = [1.0, 0.3, 120, 30, 30, 7.0, 4.0, 1.0]

    fig, ax = plt.subplots()
    x = np.arange(len(features))
    ax.bar(x - 0.2, values, width=0.4, label='Patient')
    ax.bar(x + 0.2, ref_values, width=0.4, label='Normal')
    ax.set_xticks(x)
    ax.set_xticklabels(features, rotation=45)
    ax.set_ylabel("Value")
    ax.set_title("Liver Function Test Comparison")
    ax.legend()
    plt.tight_layout()
    return fig

def generate_feature_importance():
    features = ['Age', 'Gender', 'Total Bilirubin', 'Direct Bilirubin', 'Alkaline Phosphotase', 'ALT', 'AST', 'Proteins', 'Albumin', 'A/G Ratio']
    importances = model.feature_importances_

    fig, ax = plt.subplots()
    ax.barh(features, importances)
    ax.set_xlabel("Importance")
    ax.set_title("Feature Importance - Random Forest")
    plt.tight_layout()
    return fig

def predict_and_recommend(age, gender, tb, db, alk_phos, alt, ast, proteins, albumin, ag_ratio):
    input_data = np.array([[age, gender, tb, db, alk_phos, alt, ast, proteins, albumin, ag_ratio]])
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    prob = model.predict_proba(input_scaled)[0][1]

    info = {
        'age': age, 'gender': gender, 'tb': tb, 'db': db,
        'alk_phos': alk_phos, 'alt': alt, 'ast': ast,
        'proteins': proteins, 'albumin': albumin, 'ag_ratio': ag_ratio
    }

    if prediction == 1:
        summary = (
            f"🟥 **NAFLD Detected**  \n"
            f"🔬 **Confidence:** {prob:.2%}  \n"
            f"\nYour liver profile indicates a high likelihood of Non-Alcoholic Fatty Liver Disease.\n"
            f"\n📌 Next: Please consult your doctor for confirmation and care.\n"
            f"🔍 See the \"Recommendations\" tab for lifestyle tips."
        )
    else:
        summary = (
            f"🟩 **No NAFLD Detected**  \n"
            f"✅ **Confidence:** {(1 - prob):.2%}  \n"
            f"\nYour test results do not indicate signs of NAFLD.\n"
            f"\n📌 Stay healthy: Maintain your lifestyle and consider regular checkups.\n"
            f"🔍 Visit \"Recommendations\" for proactive wellness advice."
        )

    recommendation = get_recommendation(info, prediction == 1)
    chart = generate_liver_plot(info)
    importance = generate_feature_importance()

    with tempfile.NamedTemporaryFile(delete=False, suffix=".txt", mode='w') as f:
        f.write("Liver Health Report\n")
        f.write("="*40 + "\n\n")
        f.write(summary.replace("  \n", "\n") + "\n")
        f.write(recommendation + "\n")
        filepath = f.name

    return summary, recommendation, filepath, chart, importance

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            gr.Image(value="/content/drive/MyDrive/Project/Images/LIV-Insight1.png", show_label=False, height=200, width=200)

    gr.HTML("""
        <h1 style='text-align: center; color: #2c3e50;'>🩺 <b>NAFLD Prediction & Lifestyle Recommendations App</b></h1>
        <hr>
        <div style='font-size: 16px; line-height: 1.6;'>
        🌟 <b>Welcome!</b> This app helps assess your risk of <b>Non-Alcoholic Fatty Liver Disease (NAFLD)</b> using real clinical data and an advanced <b>machine learning</b> model.
        <ul>
        <li>🎯 <b>Instant prediction</b> based on medical patterns</li>
        <li>🧠 <b>Personalized advice</b> generated by a medical LLM</li>
        <li>📥 Option to <b>download your report</b></li>
        </ul>
        🛡️ Ideal for health checks, early detection, or prep for clinical visits.
        </div>
        <hr>
    """)

    gr.Markdown("### 🔢 Patient Demographics")
    with gr.Row():
        age = gr.Number(label="Age", minimum=7, info="e.g., 45, 60, 70")
        gender = gr.Radio([0, 1], label="Gender (0 = Female, 1 = Male)", info="Select 0 for Female, 1 for Male")

    gr.Markdown("### 🧪 Liver Function Test Results")
    with gr.Row():
        tb = gr.Number(label="Total Bilirubin", info="A yellow pigment from red blood cells. Elevated levels may indicate liver dysfunction.")
        db = gr.Number(label="Direct Bilirubin", info="Conjugated bilirubin. Higher levels can reflect bile duct problems.")
    with gr.Row():
        alk_phos = gr.Number(label="Alkaline Phosphotase", info="An enzyme. Elevated in liver or bone disease.")
        alt = gr.Number(label="ALT (SGPT)", info="An enzyme found in liver. High levels may signal liver damage.")
        ast = gr.Number(label="AST (SGOT)", info="An enzyme present in liver and muscles. Elevated levels suggest liver stress.")
    with gr.Row():
        proteins = gr.Number(label="Total Proteins", info="Reflects total amount of albumin and globulin in blood.")
        albumin = gr.Number(label="Albumin", info="A protein made by the liver. Low levels can indicate liver disease.")
        ag_ratio = gr.Number(label="A/G Ratio", info="Ratio of albumin to globulin. Used to evaluate liver function.")


    with gr.Tabs():
        with gr.TabItem("🔍 Prediction with Confidence"):
            predict_btn = gr.Button("🔍 Predict NAFLD Risk")
            with gr.Accordion("📝 View Prediction Result", open=True):
                result_box = gr.Markdown()
        with gr.TabItem("🧠Medical LLM Recommendations"):
            rec_output = gr.Markdown()
        with gr.TabItem("📥 Download Report"):
            download_file = gr.File(label="Download Liver Health Report")
        with gr.TabItem("📊 Visual Insights"):
            chart_output = gr.Plot(label="📈 Bar Chart")
            importance_output = gr.Plot(label="📌 Feature Importance")

    predict_btn.click(
        predict_and_recommend,
        inputs=[age, gender, tb, db, alk_phos, alt, ast, proteins, albumin, ag_ratio],
        outputs=[result_box, rec_output, download_file, chart_output, importance_output]
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://345add7265104d7615.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
